## Problem Statements

The F&B space has witnessed massive growth in the past few years. Opportunities aside, F&B sector also operating in a competitive environment with the increase in numbers of restaurants. 
The goal of the project is to utilize user reviews data to help business owners understand why a business has good/bad ratings. One could learn, exploit and improve the weaknesses of a restaurant and thus get a business advantage. Develop a classification model based on restaurants attributes to predict if a business will be closed. This enables investors to access the risk before investing in a restaurant. 

In [1]:
# import libraries
import pandas as pd    
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import re
from sklearn.metrics import accuracy_score

from datetime import datetime

import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## Load data

In [2]:
restaurants = pd.read_pickle('./data/restaurants_data_raw.pkl')

In [3]:
# reset index
restaurants = restaurants.reset_index()

In [4]:
restaurants.head()

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
1,12,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","Food, Pizza, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
2,26,hcRxdDg7DYryCxCoI8ySQA,Longwood Galleria,340-350 Longwood Ave,Boston,MA,02215,42.338544,-71.106842,2.5,24,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Restaurants, Shopping, Shopping Centers","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
3,29,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,1,"{'NoiseLevel': 'u'average'', 'BikeParking': 'F...","Sandwiches, Food, Restaurants, Breakfast & Bru...","{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ..."
4,35,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,0,"{'NoiseLevel': ''quiet'', 'GoodForKids': 'True...","Creperies, Restaurants, Food, Coffee & Tea, Br...","{'Tuesday': '8:0-20:0', 'Wednesday': '8:0-20:0..."


In [5]:
# remove index column
restaurants.drop(labels='index',axis=1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","Food, Pizza, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
2,hcRxdDg7DYryCxCoI8ySQA,Longwood Galleria,340-350 Longwood Ave,Boston,MA,02215,42.338544,-71.106842,2.5,24,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Restaurants, Shopping, Shopping Centers","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
3,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,1,"{'NoiseLevel': 'u'average'', 'BikeParking': 'F...","Sandwiches, Food, Restaurants, Breakfast & Bru...","{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ..."
4,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,0,"{'NoiseLevel': ''quiet'', 'GoodForKids': 'True...","Creperies, Restaurants, Food, Coffee & Tea, Br...","{'Tuesday': '8:0-20:0', 'Wednesday': '8:0-20:0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18087,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,OR,97035,45.399154,-122.737600,4.0,137,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Nightlife, Wine Bars, Beer Bar, Bars, Restaura...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-20:0', ..."
18088,yQL8SrSETbbCI1U5esVJQw,Ciao! Pizza & Pasta,59 Williams St,Chelsea,MA,02150,42.389221,-71.040882,5.0,733,1,"{'GoodForKids': 'True', 'RestaurantsPriceRange...","Restaurants, Pizza, Italian","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
18089,_-nynGfhsMVVWWbAZ6YhTw,The Jury Room,39 Cottage Ave,Quincy,MA,02269,42.249118,-71.001071,3.0,30,0,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, American (New), Nightlife","{'Monday': '11:30-1:0', 'Tuesday': '11:30-1:0'..."
18090,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,45.394457,-122.752535,2.0,48,0,"{'RestaurantsDelivery': 'False', 'GoodForKids'...","American (Traditional), Desserts, American (Ne...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."


In [6]:
restaurants.shape

(18092, 15)

In [7]:
restaurants.describe()

,index,latitude,longitude,stars,review_count,is_open
count,18092.000000,18092.000000,18092.000000,18092.000000,18092.000000,18092.000000
mean,80462.817101,43.663324,-92.380952,3.596065,122.194174,0.598386
std,46167.597745,1.546812,25.426747,0.749110,237.555356,0.490238
min,1.000000,42.100171,-122.898977,1.000000,5.000000,0.000000
25%,40827.750000,42.351780,-122.651327,3.000000,19.000000,0.000000
50%,80860.500000,42.478496,-71.181757,3.500000,53.000000,1.000000
75%,119909.500000,45.508249,-71.069529,4.000000,138.000000,1.000000
max,160583.000000,45.642123,71.113271,5.000000,9185.000000,1.000000


In [8]:
restaurants.dtypes

index             int64
business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

In [9]:
user_reviews = pd.read_pickle('./data/users_review_raw.pkl')

In [10]:
user_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
5,J4a2TuhDasjn2k3wWtHZnQ,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1.0,2,0,0,"This place used to be a cool, chill place. Now...",2018-01-21 04:41:03
6,28gGfkLs3igtjVy61lh77Q,Q8c91v7luItVB0cMFF_mRA,EXOsmAB1s71WePlQk0WZrA,2.0,0,0,0,"The setting is perfectly adequate, and the foo...",2006-04-16 02:58:44
9,KKVFopqzcVfcubIBxmIjVA,99RsBrARhhx60UnAC4yDoA,EEHhKSxUvJkoPSzeGKkpVg,5.0,0,0,0,I work in the Pru and this is the most afforda...,2014-05-07 18:10:21
18,btNWW2kdJYfwpTDyzJO3Iw,DECuRZwkUw8ELQZfNGef2Q,zmZ3HkVCeZPBefJJxzdJ7A,4.0,0,0,0,Nothing special but good enough. I like anoth...,2012-12-04 04:29:47


In [11]:
# reset index
user_reviews = user_reviews.reset_index()

In [12]:
# Check missing value
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2282296 entries, 0 to 2282295
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   index        int64  
 1   review_id    object 
 2   user_id      object 
 3   business_id  object 
 4   stars        float64
 5   useful       int64  
 6   funny        int64  
 7   cool         int64  
 8   text         object 
 9   date         object 
dtypes: float64(1), int64(4), object(5)
memory usage: 174.1+ MB


## Cleaning and pre-process restaurant ['categories'] columns

In [13]:
# Identify categories and count the categories by restaurant
business_overall = restaurants.assign(categories = restaurants.categories.str.split(', ')).explode('categories')
cat = business_overall.categories.value_counts()
cat = cat.to_frame().reset_index()
cat.columns = ['Categories', 'Count']

In [14]:
cat['Categories'].unique()

array(['Restaurants', 'Food', 'Nightlife', 'Bars', 'Sandwiches', 'Pizza',
       'American (Traditional)', 'Breakfast & Brunch', 'American (New)',
       'Coffee & Tea', 'Italian', 'Mexican', 'Burgers', 'Fast Food',
       'Chinese', 'Cafes', 'Seafood', 'Event Planning & Services',
       'Salad', 'Food Trucks', 'Japanese', 'Bakeries', 'Food Stands',
       'Specialty Food', 'Asian Fusion', 'Delis', 'Thai', 'Caterers',
       'Sushi Bars', 'Cocktail Bars', 'Desserts', 'Pubs', 'Mediterranean',
       'Vegetarian', 'Vegan', 'Wine & Spirits', 'Beer', 'Gluten-Free',
       'Sports Bars', 'Chicken Wings', 'Vietnamese', 'Lounges',
       'Barbeque', 'Wine Bars', 'Soup', 'Arts & Entertainment', 'Diners',
       'Juice Bars & Smoothies', 'Ice Cream & Frozen Yogurt',
       'Middle Eastern', 'Indian', 'Steakhouses', 'Latin American',
       'Bagels', 'Korean', 'Food Delivery Services', 'Grocery',
       'Street Vendors', 'Venues & Event Spaces', 'Noodles', 'Shopping',
       'Greek', 'Beer Bar'

In [15]:
def assign_cat(label,cat):
    restaurants.loc[restaurants.categories.str.contains(label),'category'] = cat

In [16]:
# Recreate categories of restaurants
# Less significant
assign_cat('Food Stands','Streets')
assign_cat('Food Trucks','Streets')
assign_cat('Nightlife','Nightlife')
assign_cat('Cuban','Cuban')
assign_cat('Asian Fusion','Fusion')
assign_cat('Fast Food','Fast_Food')
assign_cat('Streets','Streets')


# More significant
assign_cat('Cafes','Cafes')
assign_cat('Coffee & Tea','Cafes')
assign_cat('Wine','Bars')
assign_cat('Bars','Bars')
assign_cat('Bakeries','Bakeries')
assign_cat('Pizza','Western')
assign_cat('Burgers','Western')


assign_cat('American','American')
assign_cat('Mexican','Mexican')
assign_cat('Tacos','Mexican')
assign_cat('Thai','Thai')
assign_cat('Japanese','Japanese')
assign_cat('Sushi','Japanese')
assign_cat('Vegetarian','Vegetarian/Vegan')
assign_cat('Vegan','Vegetarian/Vegan')
assign_cat('Italian','Italian')
assign_cat('Chinese','Chinese')
assign_cat('Mediterranean','Mediterranean')
assign_cat('French','French')
assign_cat('Vietnamese','Vietnamese')
assign_cat('Greek','Greek')
assign_cat('Indian','Indian')
assign_cat('Korean','Korean')
assign_cat('Hawaiian','Hawaiian')
assign_cat('African','African')
assign_cat('Spanish','Spanish')
assign_cat('German','German')
assign_cat('Middle_eastern','Middle_eastern')
assign_cat('Filipino','Filipino')


In [17]:
restaurants['category'].isnull().sum()

1836

In [18]:
# Drop categories
restaurants.drop(columns='categories',inplace=True)

In [19]:
restaurants[restaurants['category'].isnull()]

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours,category
2,26,hcRxdDg7DYryCxCoI8ySQA,Longwood Galleria,340-350 Longwood Ave,Boston,MA,02215,42.338544,-71.106842,2.5,24,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'...",NaN
9,68,G6xF2EGY-F04iFhq5sX8MQ,MJB Restoration,,Portland,OR,97213,45.535283,-122.603754,4.5,9,1,{'ByAppointmentOnly': 'True'},None,NaN
20,187,fDJm1OQWJg6XyuVTcxoemg,The Barrel,613 Main St,Hingham,MA,02043,42.215607,-70.885132,3.5,11,1,"{'BusinessParking': '{'garage': False, 'street...","{'Monday': '6:30-20:0', 'Tuesday': '6:30-20:0'...",NaN
23,197,EJ58GOWZplYJkj4TkDuSXw,D'angelo Sandwich Shop,1277 Broadway,Saugus,MA,01906,42.461100,-71.026910,2.0,6,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsDel...",None,NaN
27,221,fD9EopsUWvimsi4eaS7uMA,The Wheelhouse Diner,453 Hancock St,Quincy,MA,02171,42.272234,-71.023950,4.0,229,1,"{'HasTV': 'False', 'BusinessParking': '{'garag...","{'Wednesday': '5:0-15:0', 'Thursday': '5:0-11:...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18034,160015,bkv4EZ5RYRvZITO1PaOaNg,Best Baguette,1833 SW 6th Ave,Portland,OR,97201,45.511406,-122.682940,3.5,47,1,"{'GoodForKids': 'False', 'DogsAllowed': 'False...",None,NaN
18041,160133,jHg7ZDzcxV2mYyf7XThk9w,King Crab Juicy Seafood,180 Main St,Saugus,MA,01906,42.463541,-71.026098,3.5,15,1,"{'WheelchairAccessible': 'True', 'RestaurantsD...","{'Monday': '0:0-0:0', 'Tuesday': '11:30-22:0',...",NaN
18059,160378,znkU8_QJ4Dksd8XUVjXBcg,Fuku Boston Seaport,43 Northern Ave,Boston,MA,02210,42.353272,-71.046872,3.5,182,0,"{'RestaurantsReservations': 'False', 'HappyHou...","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",NaN
18066,160455,nS8uRoVeqe20VsDDH59jHQ,Boston Foodie Tours,,Boston,MA,02114,42.365292,-71.064634,5.0,67,1,"{'BusinessParking': '{'garage': True, 'street'...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",NaN


In [20]:
# Drop 1836 rows null category
restaurants.drop(restaurants[restaurants['category'].isnull()].index, inplace=True)

In [21]:
restaurants.drop(labels='index', axis=1, inplace=True)

In [22]:
restaurants.reset_index(inplace=True)

In [23]:
restaurants.drop(labels='index', axis=1, inplace=True)

In [24]:
restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours,category
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",Vegetarian/Vegan
1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",Western
2,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,1,"{'NoiseLevel': 'u'average'', 'BikeParking': 'F...","{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",Italian
3,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,0,"{'NoiseLevel': ''quiet'', 'GoodForKids': 'True...","{'Tuesday': '8:0-20:0', 'Wednesday': '8:0-20:0...",Cafes
4,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits,12429 NE Glisan St,Portland,OR,97230,45.526473,-122.535323,3.5,19,1,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",Nightlife
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,OR,97035,45.399154,-122.737600,4.0,137,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-20:0', ...",American
16252,yQL8SrSETbbCI1U5esVJQw,Ciao! Pizza & Pasta,59 Williams St,Chelsea,MA,02150,42.389221,-71.040882,5.0,733,1,"{'GoodForKids': 'True', 'RestaurantsPriceRange...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",Italian
16253,_-nynGfhsMVVWWbAZ6YhTw,The Jury Room,39 Cottage Ave,Quincy,MA,02269,42.249118,-71.001071,3.0,30,0,"{'BusinessParking': '{'garage': False, 'street...","{'Monday': '11:30-1:0', 'Tuesday': '11:30-1:0'...",American
16254,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,45.394457,-122.752535,2.0,48,0,"{'RestaurantsDelivery': 'False', 'GoodForKids'...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",American


## Drop hours columns

In [25]:
# Drop hours
restaurants.drop(columns='hours', axis=1,inplace=True)

## Clean and preprocess `attributes` columns

In [26]:
# break up dict inside `attributes` columns
df_attr = restaurants['attributes'].apply(pd.Series)
df_attr

,RestaurantsTakeOut,RestaurantsAttire,GoodForKids,BikeParking,OutdoorSeating,Ambience,Caters,RestaurantsReservations,RestaurantsDelivery,HasTV,...,Corkage,BYOB,CoatCheck,Smoking,DriveThru,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,Open24Hours,HairSpecializesIn
0,True,u'casual',True,False,False,"{'romantic': False, 'intimate': False, 'touris...",True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,u'casual',True,True,True,"{'romantic': False, 'intimate': False, 'classy...",True,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,u'casual',True,False,False,"{'touristy': None, 'hipster': False, 'romantic...",False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,'casual',True,True,True,"{'romantic': False, 'intimate': False, 'classy...",True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,'casual',True,True,False,"{'romantic': False, 'intimate': False, 'touris...",NaN,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,True,'casual',True,True,True,"{'romantic': False, 'intimate': False, 'classy...",True,False,True,True,...,NaN,NaN,False,u'no',NaN,NaN,NaN,NaN,NaN,NaN
16252,True,u'casual',True,False,True,"{'touristy': False, 'hipster': False, 'romanti...",False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16253,True,'casual',False,NaN,False,NaN,NaN,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16254,False,'casual',False,NaN,True,"{'romantic': False, 'intimate': False, 'touris...",False,True,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Calculate the missing percentage for the attrbutes feature
percent_missing = df_attr.isnull().sum() * 100 / len(df_attr)
missing_value_df = pd.DataFrame({'column_name': df_attr.columns,
                                 'percent_missing': percent_missing})

In [28]:
missing_value_df.sort_values('percent_missing', inplace=True)

In [29]:
missing_value_df

,column_name,percent_missing
RestaurantsTakeOut,RestaurantsTakeOut,5.031988
BusinessAcceptsCreditCards,BusinessAcceptsCreditCards,5.038140
RestaurantsDelivery,RestaurantsDelivery,6.889764
BusinessParking,BusinessParking,7.092766
RestaurantsPriceRange2,RestaurantsPriceRange2,9.781004
OutdoorSeating,OutdoorSeating,10.113189
RestaurantsReservations,RestaurantsReservations,12.475394
RestaurantsGoodForGroups,RestaurantsGoodForGroups,14.788386
Alcohol,Alcohol,15.668061
GoodForKids,GoodForKids,15.828002


In [30]:
## Drop restaurant attributes which has more than 30% missing values
attr_drop = list(missing_value_df[missing_value_df['percent_missing'] >= 30].index)
df_attr = df_attr.drop(columns=attr_drop, axis=1)

In [31]:
# Create functions to extract the dicts stored in attributes, return dataframe with dic keys as columns
    
def str_dict_to_df(series):

    eval_list = []
    for sr in series:
        if not pd.isna(sr):
            eval_list.append(eval(sr))
        else:
            eval_list.append(np.nan)
    
    eval_df = pd.Series(eval_list).apply(pd.Series)  
    
    #drop cols that are all nan
    eval_df = eval_df.dropna(axis=1, how='all')
    
    
    return eval_df

In [32]:
#store col names
dict_cols_list = []

for col in df_attr:
    #if contains a curly bracket, then assume column is a dict as string
    if df_attr[col].str.contains('{').any():
        dict_cols_list.append(col)
print(dict_cols_list)   

['Ambience', 'BusinessParking']


In [33]:
#store dataframes from dict nested columns
dict_col_df_list = []

for col in dict_cols_list:
    #apply string to dict evaluation
    temp_df = str_dict_to_df(df_attr[col])
    #append to list
    dict_col_df_list.append(temp_df)

#combine all in one column
dict_col_df = pd.concat(dict_col_df_list, axis=1)
dict_col_df.head()

,casual,classy,divey,hipster,intimate,romantic,touristy,trendy,upscale,garage,lot,street,valet,validated
0,True,False,False,False,False,False,False,False,False,True,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,True,False,False,False
2,True,True,False,False,False,False,None,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,True,False,False,False


In [34]:
dict_col_df.shape

(16256, 14)

In [35]:
# join the dict_col_df to df_attr, drop ['BusinessParking'] & ['Ambience']
df_attr_conc = pd.concat([df_attr, dict_col_df], axis =1)
df_attr_conc.drop(columns=['BusinessParking','Ambience'],axis=1, inplace = True)

In [36]:
df_attr_conc

,RestaurantsTakeOut,RestaurantsAttire,GoodForKids,BikeParking,OutdoorSeating,Caters,RestaurantsReservations,RestaurantsDelivery,HasTV,RestaurantsGoodForGroups,...,intimate,romantic,touristy,trendy,upscale,garage,lot,street,valet,validated
0,True,u'casual',True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,True,u'casual',True,True,True,True,False,True,True,True,...,False,False,False,False,False,False,True,False,False,False
2,True,u'casual',True,False,False,False,False,False,True,False,...,False,False,None,False,False,True,False,False,False,False
3,True,'casual',True,True,True,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,True,'casual',True,True,False,NaN,False,False,True,True,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,True,'casual',True,True,True,True,False,True,True,True,...,False,False,False,False,False,False,True,True,False,False
16252,True,u'casual',True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
16253,True,'casual',False,NaN,False,NaN,True,False,NaN,True,...,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False
16254,False,'casual',False,NaN,True,False,True,False,True,True,...,False,False,False,False,False,True,False,False,True,False


In [37]:
df_attr_conc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16256 entries, 0 to 16255
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   RestaurantsTakeOut          15438 non-null  object
 1   RestaurantsAttire           13301 non-null  object
 2   GoodForKids                 13683 non-null  object
 3   BikeParking                 11586 non-null  object
 4   OutdoorSeating              14612 non-null  object
 5   Caters                      11560 non-null  object
 6   RestaurantsReservations     14228 non-null  object
 7   RestaurantsDelivery         15136 non-null  object
 8   HasTV                       13467 non-null  object
 9   RestaurantsGoodForGroups    13852 non-null  object
 10  BusinessAcceptsCreditCards  15437 non-null  object
 11  NoiseLevel                  11903 non-null  object
 12  RestaurantsPriceRange2      14666 non-null  object
 13  WiFi                        12718 non-null  ob

In [38]:
df_attr_conc.columns

Index(['RestaurantsTakeOut', 'RestaurantsAttire', 'GoodForKids', 'BikeParking',
       'OutdoorSeating', 'Caters', 'RestaurantsReservations',
       'RestaurantsDelivery', 'HasTV', 'RestaurantsGoodForGroups',
       'BusinessAcceptsCreditCards', 'NoiseLevel', 'RestaurantsPriceRange2',
       'WiFi', 'Alcohol', 'casual', 'classy', 'divey', 'hipster', 'intimate',
       'romantic', 'touristy', 'trendy', 'upscale', 'garage', 'lot', 'street',
       'valet', 'validated'],
      dtype='object')

### Replace 'None' with np.NaN

In [39]:
df_attr_conc.replace({'True': True, 'False':False, 'None': np.NaN}, inplace = True)

### Clean the attributes that begins with u'XXX'

In [40]:
for col in df_attr_conc:
    print(col)
    print(df_attr_conc[col].unique())
    print(50*'-')

RestaurantsTakeOut
[True False nan]
--------------------------------------------------
RestaurantsAttire
["u'casual'" "'casual'" nan "u'formal'" "u'dressy'" "'dressy'" "'formal'"]
--------------------------------------------------
GoodForKids
[True False nan]
--------------------------------------------------
BikeParking
[False True nan]
--------------------------------------------------
OutdoorSeating
[False True nan]
--------------------------------------------------
Caters
[True False nan]
--------------------------------------------------
RestaurantsReservations
[False True nan]
--------------------------------------------------
RestaurantsDelivery
[False True nan]
--------------------------------------------------
HasTV
[False True nan]
--------------------------------------------------
RestaurantsGoodForGroups
[False True nan]
--------------------------------------------------
BusinessAcceptsCreditCards
[True nan False]
--------------------------------------------------
NoiseLeve

In [41]:
df_attr_conc

,RestaurantsTakeOut,RestaurantsAttire,GoodForKids,BikeParking,OutdoorSeating,Caters,RestaurantsReservations,RestaurantsDelivery,HasTV,RestaurantsGoodForGroups,...,intimate,romantic,touristy,trendy,upscale,garage,lot,street,valet,validated
0,True,u'casual',True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,True,u'casual',True,True,True,True,False,True,True,True,...,False,False,False,False,False,False,True,False,False,False
2,True,u'casual',True,False,False,False,False,False,True,False,...,False,False,None,False,False,True,False,False,False,False
3,True,'casual',True,True,True,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,True,'casual',True,True,False,NaN,False,False,True,True,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,True,'casual',True,True,True,True,False,True,True,True,...,False,False,False,False,False,False,True,True,False,False
16252,True,u'casual',True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
16253,True,'casual',False,NaN,False,NaN,True,False,NaN,True,...,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False
16254,False,'casual',False,NaN,True,False,True,False,True,True,...,False,False,False,False,False,True,False,False,True,False


In [42]:
#store dummy attribute columns in list
atr_dum_df_cols = []

#iterate over object columns
for col in df_attr_conc:
    #if contains a categorical variable then it begins with a u
    try:
        if df_attr_conc[col].str.contains("u'").any():
            #add to the list
            atr_dum_df_cols.append(col)
    except:
        pass

#print out list
print(atr_dum_df_cols)

['RestaurantsAttire', 'NoiseLevel', 'WiFi', 'Alcohol']


In [43]:
df_attr_conc['RestaurantsAttire'].unique()

array(["u'casual'", "'casual'", nan, "u'formal'", "u'dressy'", "'dressy'",
       "'formal'"], dtype=object)

In [44]:
# Restaurant Attire / formality
restaurant_attire_dict = dict(zip(
    ["u'casual'", "u'dressy'", "u'formal'", 'None',"'casual'","'dressy'","'formal'"],
    ['casual','dressy','formal',np.NaN,'casual','dressy','formal']))

print(restaurant_attire_dict)

{"u'casual'": 'casual', "u'dressy'": 'dressy', "u'formal'": 'formal', 'None': nan, "'casual'": 'casual', "'dressy'": 'dressy', "'formal'": 'formal'}


In [45]:
#Noise Level
noise_level_dict = dict(zip(
    ["u'average'", np.nan, "'average'", "u'quiet'", "u'loud'", "u'very_loud'",
 "'quiet'", "'very_loud'", "'loud'" ,'None'],
    [2,0,2,1,3,4,1,4,3,0]))

print(noise_level_dict)

{"u'average'": 2, nan: 0, "'average'": 2, "u'quiet'": 1, "u'loud'": 3, "u'very_loud'": 4, "'quiet'": 1, "'very_loud'": 4, "'loud'": 3, 'None': 0}


In [46]:
#WiFI
wifi_dict = dict(zip(
    ["u'free'", "'free'", "u'no'", "'no'", "u'paid'", "'paid'"],
    ['Free','Free','No','No','Paid', 'Paid']))

print(wifi_dict )

{"u'free'": 'Free', "'free'": 'Free', "u'no'": 'No', "'no'": 'No', "u'paid'": 'Paid', "'paid'": 'Paid'}


In [47]:
df_attr_conc['Alcohol'].unique()

array(["u'beer_and_wine'", "u'none'", "u'full_bar'", "'none'", nan,
       "'beer_and_wine'", "'full_bar'"], dtype=object)

In [48]:
#Alcohol
alcohol_dict = dict(zip(
    ["u'beer_and_wine'", "u'none'", "u'full_bar'",
       "'none'","'beer_and_wine'","'full_bar'"],
    ['beer_and_wine',np.NaN,'full_bar',np.NaN,'beer_and_wine','full_bar']))

print(alcohol_dict)

{"u'beer_and_wine'": 'beer_and_wine', "u'none'": nan, "u'full_bar'": 'full_bar', "'none'": nan, "'beer_and_wine'": 'beer_and_wine', "'full_bar'": 'full_bar'}


In [49]:
#define mapping for replace method
atr_dict_map = {'RestaurantsAttire': restaurant_attire_dict,
               'NoiseLevel':noise_level_dict,
               'WiFi':wifi_dict,
               'Alcohol':alcohol_dict}

In [50]:
# get a copy with relevant columns
df_atr_categorical = df_attr[atr_dum_df_cols].copy()


#apply replacement
df_atr_categorical = df_atr_categorical.replace(atr_dict_map)

df_atr_categorical

,RestaurantsAttire,NoiseLevel,WiFi,Alcohol
0,casual,2,Free,beer_and_wine
1,casual,2,Free,NaN
2,casual,2,Free,full_bar
3,casual,1,NaN,NaN
4,casual,2,No,full_bar
...,...,...,...,...
16251,casual,2,Free,beer_and_wine
16252,casual,2,No,beer_and_wine
16253,casual,0,NaN,full_bar
16254,casual,2,No,full_bar


### Impute missing value of Restaurants price range with 0

df['mix_col'] = pd.to_numeric(
    df['mix_col'], 
    errors='coerce'
).fillna(0).astype('int')

In [51]:
df_attr_conc['RestaurantsPriceRange2'] = pd.to_numeric(df_attr_conc['RestaurantsPriceRange2'],errors='coerce').fillna(0).astype('int') 

In [52]:
df_attr_conc['RestaurantsPriceRange2'].value_counts()

2    7206
1    6746
0    1591
3     627
4      86
Name: RestaurantsPriceRange2, dtype: int64

In [53]:
df_attr_conc['RestaurantsPriceRange2'].replace(0, df_attr_conc['RestaurantsPriceRange2'].median(), inplace=True )

In [54]:
df_attr_conc

,RestaurantsTakeOut,RestaurantsAttire,GoodForKids,BikeParking,OutdoorSeating,Caters,RestaurantsReservations,RestaurantsDelivery,HasTV,RestaurantsGoodForGroups,...,intimate,romantic,touristy,trendy,upscale,garage,lot,street,valet,validated
0,True,u'casual',True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,True,u'casual',True,True,True,True,False,True,True,True,...,False,False,False,False,False,False,True,False,False,False
2,True,u'casual',True,False,False,False,False,False,True,False,...,False,False,None,False,False,True,False,False,False,False
3,True,'casual',True,True,True,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,True,'casual',True,True,False,NaN,False,False,True,True,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,True,'casual',True,True,True,True,False,True,True,True,...,False,False,False,False,False,False,True,True,False,False
16252,True,u'casual',True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
16253,True,'casual',False,NaN,False,NaN,True,False,NaN,True,...,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False
16254,False,'casual',False,NaN,True,False,True,False,True,True,...,False,False,False,False,False,True,False,False,True,False


In [55]:
# drop ['RestaurantsAttire', 'NoiseLevel', 'WiFi', 'Alcohol'] in df_attr_conc
df_attr_conc.drop(columns=['RestaurantsAttire', 'NoiseLevel', 'WiFi', 'Alcohol'],axis=1, inplace = True)

In [56]:
# combine cleaned attribute as df_attr_cleaned

df_attr_cleaned = pd.concat([df_attr_conc, df_atr_categorical], axis =1)

In [57]:
#key for reference
df_attr_cleaned['business_id'] = restaurants['business_id']

In [59]:
df_attr_cleaned.columns

Index(['RestaurantsTakeOut', 'GoodForKids', 'BikeParking', 'OutdoorSeating',
       'Caters', 'RestaurantsReservations', 'RestaurantsDelivery', 'HasTV',
       'RestaurantsGoodForGroups', 'BusinessAcceptsCreditCards',
       'RestaurantsPriceRange2', 'casual', 'classy', 'divey', 'hipster',
       'intimate', 'romantic', 'touristy', 'trendy', 'upscale', 'garage',
       'lot', 'street', 'valet', 'validated', 'RestaurantsAttire',
       'NoiseLevel', 'WiFi', 'Alcohol', 'business_id'],
      dtype='object')

In [60]:
restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,category
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...",Vegetarian/Vegan
1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...",Western
2,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,1,"{'NoiseLevel': 'u'average'', 'BikeParking': 'F...",Italian
3,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,0,"{'NoiseLevel': ''quiet'', 'GoodForKids': 'True...",Cafes
4,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits,12429 NE Glisan St,Portland,OR,97230,45.526473,-122.535323,3.5,19,1,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...",Nightlife
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,OR,97035,45.399154,-122.737600,4.0,137,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...",American
16252,yQL8SrSETbbCI1U5esVJQw,Ciao! Pizza & Pasta,59 Williams St,Chelsea,MA,02150,42.389221,-71.040882,5.0,733,1,"{'GoodForKids': 'True', 'RestaurantsPriceRange...",Italian
16253,_-nynGfhsMVVWWbAZ6YhTw,The Jury Room,39 Cottage Ave,Quincy,MA,02269,42.249118,-71.001071,3.0,30,0,"{'BusinessParking': '{'garage': False, 'street...",American
16254,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,45.394457,-122.752535,2.0,48,0,"{'RestaurantsDelivery': 'False', 'GoodForKids'...",American


In [61]:
# Drop 'attributes'
restaurants.drop(columns='attributes',axis=1,inplace=True)

## Restaurant Chains

Conduct additional feature engineering to improve the model.
Is the business part of a chain.

In [62]:
restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,category
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,Vegetarian/Vegan
1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,Western
2,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,1,Italian
3,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,0,Cafes
4,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits,12429 NE Glisan St,Portland,OR,97230,45.526473,-122.535323,3.5,19,1,Nightlife


In [63]:
#get all names that are lower and duplicated
name_lower = restaurants['name'].str.lower().copy()

In [64]:
name_val_count = name_lower.value_counts(sort=True, ascending=False)

In [65]:
#see very very popular chains
name_val_count[:30]

subway                           174
mcdonald's                       138
dunkin'                           86
burger king                       70
domino's pizza                    70
starbucks                         68
taco bell                         56
chipotle mexican grill            55
wendy's                           46
papa gino's pizzeria              39
panera bread                      34
pizza hut                         33
papa murphy's                     31
kfc                               28
panda express                     26
five guys                         26
au bon pain                       24
99 restaurants                    22
b.good                            21
bertucci's italian restaurant     19
qdoba mexican eats                19
jimmy john's                      18
burgerville                       18
papa john's pizza                 17
sweetgreen                        16
jack in the box                   16
comella's                         15
p

In [66]:
#see even less popular chains
name_val_count[(name_val_count.values==3)]

petsi pies               3
chacarero                3
aquitaine                3
teriyaki house           3
jim & patty's coffee     3
                        ..
round table clubhouse    3
cha! cha! cha!           3
the real deal            3
the upper crust          3
brother's roast beef     3
Name: name, Length: 182, dtype: int64

In [67]:
# Let chain be restaurants that has at least 5 identical entries

#get chain names
chain_names = name_val_count[(name_val_count.values>=5)].index.tolist()
#see samples
chain_names[:10]

['subway',
 "mcdonald's",
 "dunkin'",
 'burger king',
 "domino's pizza",
 'starbucks',
 'taco bell',
 'chipotle mexican grill',
 "wendy's",
 "papa gino's pizzeria"]

In [68]:
def get_chain(bus_name, chain_names=chain_names):
    #convert to string in case not a string
    bus_name_lower = str(bus_name).lower()
    #check if contained in predefied list
    if bus_name_lower in chain_names:
        return 1
    else:
        return 0
    

In [69]:
restaurants['chain'] = restaurants['name'].apply(lambda x: get_chain(x, chain_names=chain_names))

In [70]:
restaurants[restaurants['chain'] == 1]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,category,chain
2,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,1,Italian,1
5,5PsApTz-WqhF_-8mfqGtGw,Cafe Yumm!,301 SW Morrison St,Portland,OR,97204,45.518450,-122.675723,3.5,158,1,Vegetarian/Vegan,1
7,buTr0hMiyDbjQvFJgm-Eew,JP Licks,611 Assembly Row,Somerville,MA,02145,42.395406,-71.079491,3.5,131,1,Cafes,1
22,HVpIq8w2SMZK9MQT9u3yng,Gourmet Garden,139 Endicott St,Danvers,MA,01923,42.551686,-70.933121,3.5,10,1,Chinese,1
25,FRB3u9Y2boOGDjyQQo7UgA,Regina Pizzeria,"Arsenal Mall, 485 Arsenal Street",Watertown,MA,02472,42.361382,-71.158536,4.0,10,0,Italian,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16178,dwLF6jEC5UB8hPa0r93Fxg,Capital One Café,425 Revolution Dr,Somerville,MA,02145,42.391961,-71.078251,4.5,20,1,Cafes,1
16179,ieIMK7tAq9GuqQRzqM-xZA,McDonald's,180 Endicott St,Danvers,MA,01923,42.547904,-70.941543,2.5,20,1,Western,1
16189,6uuWg3RYUYzrLePphT3PQw,TGI Fridays,1626 Tremont St,Boston,MA,02120,42.333407,-71.104246,2.0,228,1,American,1
16207,e8AVhjfCjfu4Hs9yU14O-A,Subway,9226 SE Division St,Portland,OR,97266,45.504123,-122.567525,2.0,5,0,Fast_Food,1


In [71]:
#distribution among target value
restaurants.groupby(by='is_open', as_index=True).mean()[['chain']]

,chain
is_open,
0,0.080099
1,0.167433


### Merge the restaurant dataset with df_attr_cleaned

In [72]:
restaurants_cleaned = restaurants.merge(df_attr_cleaned, on = 'business_id', how='left')

In [73]:
restaurants_cleaned

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,upscale,garage,lot,street,valet,validated,RestaurantsAttire,NoiseLevel,WiFi,Alcohol
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,...,False,True,False,False,False,False,casual,2,Free,beer_and_wine
1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,...,False,False,True,False,False,False,casual,2,Free,NaN
2,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,...,False,True,False,False,False,False,casual,2,Free,full_bar
3,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,...,False,False,False,False,False,False,casual,1,NaN,NaN
4,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits,12429 NE Glisan St,Portland,OR,97230,45.526473,-122.535323,3.5,19,...,False,False,True,False,False,False,casual,2,No,full_bar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,OR,97035,45.399154,-122.737600,4.0,137,...,False,False,True,True,False,False,casual,2,Free,beer_and_wine
16252,yQL8SrSETbbCI1U5esVJQw,Ciao! Pizza & Pasta,59 Williams St,Chelsea,MA,02150,42.389221,-71.040882,5.0,733,...,False,False,True,False,False,False,casual,2,No,beer_and_wine
16253,_-nynGfhsMVVWWbAZ6YhTw,The Jury Room,39 Cottage Ave,Quincy,MA,02269,42.249118,-71.001071,3.0,30,...,NaN,False,False,True,False,False,casual,0,NaN,full_bar
16254,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,45.394457,-122.752535,2.0,48,...,False,True,False,False,True,False,casual,2,No,full_bar


### Add sentiment score to restaurants datasets

In [74]:
sentiment_score = pd.read_csv('./data/sentiment_score.csv')

In [75]:
sentiment_score

,business_id,sentiment_final
0,--6COJIAjkQwSUZci_4PJQ,0.594357
1,--UNNdnHRhsyFUbDgumdtQ,0.417455
2,-00d-Qb0q2TcWn-8LBHDZg,-0.273694
3,-0Gbsd7ztvTyFpl7jF0DIw,0.297091
4,-0iqnv7MjKrgh7Q7bYRlUQ,-0.000902
...,...,...
18088,zyNQhunb1mcSUUbnqVcU1w,-0.148889
18089,zyauuvAYdVweBK4L7wBRmw,-0.118130
18090,zzO0rjxjVAutcqFnI4VvAg,-0.483112
18091,zzcdycb7S42VnnZkwE4yNA,-0.728696


### Join the sentiment_final score, to restaurants dataframe

In [76]:
restaurants_cleaned = restaurants_cleaned.merge(sentiment_score, on = 'business_id', how='left')

In [77]:
restaurants_cleaned

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,garage,lot,street,valet,validated,RestaurantsAttire,NoiseLevel,WiFi,Alcohol,sentiment_final
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,...,True,False,False,False,False,casual,2,Free,beer_and_wine,0.366400
1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,...,False,True,False,False,False,casual,2,Free,NaN,0.132592
2,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,...,True,False,False,False,False,casual,2,Free,full_bar,0.090394
3,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,...,False,False,False,False,False,casual,1,NaN,NaN,0.655234
4,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits,12429 NE Glisan St,Portland,OR,97230,45.526473,-122.535323,3.5,19,...,False,True,False,False,False,casual,2,No,full_bar,-0.114162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,cgNDiWCaSlqqxx1A6r65bA,Hop N Cork,17450 Lower Boones Ferry Rd,Lake Oswego,OR,97035,45.399154,-122.737600,4.0,137,...,False,True,True,False,False,casual,2,Free,beer_and_wine,0.583473
16252,yQL8SrSETbbCI1U5esVJQw,Ciao! Pizza & Pasta,59 Williams St,Chelsea,MA,02150,42.389221,-71.040882,5.0,733,...,False,True,False,False,False,casual,2,No,beer_and_wine,0.676805
16253,_-nynGfhsMVVWWbAZ6YhTw,The Jury Room,39 Cottage Ave,Quincy,MA,02269,42.249118,-71.001071,3.0,30,...,False,False,True,False,False,casual,0,NaN,full_bar,-0.182383
16254,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,45.394457,-122.752535,2.0,48,...,True,False,False,True,False,casual,2,No,full_bar,-0.631424


In [78]:
restaurants_cleaned.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'category',
       'chain', 'RestaurantsTakeOut', 'GoodForKids', 'BikeParking',
       'OutdoorSeating', 'Caters', 'RestaurantsReservations',
       'RestaurantsDelivery', 'HasTV', 'RestaurantsGoodForGroups',
       'BusinessAcceptsCreditCards', 'RestaurantsPriceRange2', 'casual',
       'classy', 'divey', 'hipster', 'intimate', 'romantic', 'touristy',
       'trendy', 'upscale', 'garage', 'lot', 'street', 'valet', 'validated',
       'RestaurantsAttire', 'NoiseLevel', 'WiFi', 'Alcohol',
       'sentiment_final'],
      dtype='object')

In [79]:
#save the cleaned dataset
restaurants_cleaned.to_pickle('./data/restaurants_cleaned.pkl')

## Clean user reviews dataset

### Convert date to datetime data type

In [83]:
# convert 'date' to datetime
user_reviews['date'] = pd.to_datetime(user_reviews['date'], format = '%Y/%m/%d')

In [84]:
# create year columns
user_reviews['year'] = user_reviews.date.dt.year

In [85]:
#save the cleaned user reviews dataset
user_reviews.to_pickle('./data/user_reviews_cleaned.pkl')